# Laborator 1

In [1]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

import random

import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abuinoschi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Collect words from a given domain.

In [2]:
domain_choice = input("Choose a domain for the crossword: ")
print('You chose domain {0}.'.format(domain_choice))
domain_choice = domain_choice.lower()

Choose a domain for the crossword:  science


You chose domain science.


In [3]:
def get_hyponyms(synset):
    hyponyms = set()
    for hyponym in synset.hyponyms():
        hyponyms |= set(get_hyponyms(hyponym))
    return hyponyms | set(synset.hyponyms())

In [4]:
domain_choice_synset = wn.synset(domain_choice+'.n.1')
synset_words = list(get_hyponyms(domain_choice_synset))

synset_words_dict = {}
for synset in synset_words:
    name = synset.name().split('.')[0].replace('_', ' ')
    synset_words_dict[name] = synset
    
print(synset_words_dict)

{'psephology': Synset('psephology.n.01'), 'derivational morphology': Synset('derivational_morphology.n.01'), 'selenology': Synset('selenology.n.01'), 'electrostatics': Synset('electrostatics.n.01'), 'pteridology': Synset('pteridology.n.01'), 'climatology': Synset('climatology.n.01'), 'regional anatomy': Synset('regional_anatomy.n.01'), 'vertebrate paleontology': Synset('vertebrate_paleontology.n.01'), 'catoptrics': Synset('catoptrics.n.01'), 'macrobiotics': Synset('macrobiotics.n.01'), 'botany': Synset('botany.n.02'), 'histology': Synset('histology.n.01'), 'virology': Synset('virology.n.01'), 'trigonometry': Synset('trigonometry.n.01'), 'paleoclimatology': Synset('paleoclimatology.n.01'), 'cybernetics': Synset('cybernetics.n.01'), 'topology': Synset('topology.n.01'), 'morphology': Synset('morphology.n.04'), 'agrobiology': Synset('agrobiology.n.01'), 'egyptology': Synset('egyptology.n.01'), 'quantum field theory': Synset('quantum_field_theory.n.01'), 'herpetology': Synset('herpetology.n

In [5]:
column_word = random.choice(list(synset_words_dict.keys()))
column_synset = synset_words_dict[column_word]
while len(column_word) < 12:
    column_word = random.choice(list(synset_words_dict.keys()))
    column_synset = synset_words_dict[column_word]
    
column_clue = column_synset.definition()[0].upper() + column_synset.definition()[1:] + '.'

print(column_word)
print(column_clue)

molecular biology
The branch of biology that studies the structure and activity of macromolecules essential to life (and especially with their genetic role).


## By default the clue is set to be the definition. If there are synonyms and antonyms found, then use them as clues.

In [6]:
def choose_word_from_hyponyms(synset_dict: dict, character: str, type='definition'):
    synonym = ''
    antonym = ''
    for word in synset_dict.keys():
        if character in word:
            # By default, the clue is the definition
            clue = synset_dict[word].definition()[0].upper() + synset_dict[word].definition()[1:] + '.'
            # Search for synonym
            synonyms_syn = synset_dict[word].lemmas()
            for lemma in synonyms_syn:
                if lemma.name().lower().replace('_', ' ') != word:
                        synonym = lemma.name().replace('_', ' ')
            # Search for antonym
            synonyms_syn = synset_dict[word].lemmas()
            for lemma in synonyms_syn:
                if lemma.antonyms():
                    if lemma.antonyms()[0].name().lower().replace('_', ' ') != word:
                        antonym = lemma.antonyms()[0].name().replace('_', ' ')
            synset_dict.pop(word)
            word = word.replace(character, character.upper())
            if antonym != '':
                clue = 'Give an antonym for {0}.'.format(antonym)
            if synonym != '':
                clue = 'Find a synonym for {0}.'.format(synonym)
            word = word.lower()
            return synset_dict, word, clue   

## Generate the crosswords puzzle

In [7]:
indexes_to_fill = list(range(0, len(column_word)))
words = []
clues = []
left_size = 0
right_size = 0

for number in range(0, len(column_word)):
    synset_words_dict, word, clue = choose_word_from_hyponyms(synset_words_dict, column_word[number])
    index_meeting_column = word.lower().find(column_word[number])
    if left_size < index_meeting_column + 1:
        left_size = index_meeting_column + 1
    if right_size < len(column_word) - index_meeting_column + 1:
        right_size = len(column_word) - index_meeting_column + 1
    words.append(word.lower())
    clues.append(clue)
    
# These are important for the size of the grid
print(left_size, right_size)

print('\nThe words solutions and the clues are:')
print(words)
print(clues)

16 18

The words solutions and the clues are:
['derivational morphology', 'psephology', 'selenology', 'electrostatics', 'climatology', 'quantum field theory', 'pteridology', 'regional anatomy', 'vertebrate paleontology', 'structural genomics', 'macrobiotics', 'catoptrics', 'botany', 'histology', 'virology', 'trigonometry', 'paleoclimatology']
['The part of grammar that deals with the derivations of words.', 'The branch of sociology that studies election trends (as by opinion polls).', 'The branch of astronomy that deals with the moon.', 'The branch of physics that deals with static electricity.', 'Meteorology of climates and their phenomena.', 'The branch of quantum physics that is concerned with the theory of fields; it was motivated by the question of how an atom radiates light as its electrons jump from excited states.', 'The branch of botany that studies ferns.', 'Find a synonym for topology.', 'The paleontology of vertebrates.', 'The branch of genomics that determines the three-di

In [8]:
print("   " * (left_size + 2) + ' 0.')
for index in range(0, len(words)):
    character = column_word[index]
    word = words[index].lower()
    meeting_center_index = word.find(character)
    if index < 10:
        row_representation = "   " * (left_size - meeting_center_index + 1) + ' ' + str(index + 1) + '.'
    else: 
        row_representation = "   " * (left_size - meeting_center_index + 1) + str(index + 1) + '.'
    for letter in word.upper():
        row_representation +='|{0}|'.format(letter)
    print(row_representation)
    
print('\n\n')
print('Clues:')
print('0. ' + column_clue)
for index in range (1, len(clues)):
    print('{0}. {1}'.format(str(index), clues[index-1]))
print('{0}. {1}'.format(len(clues), clues[len(clues)-1]))

                                                       0.
             1.|D||E||R||I||V||A||T||I||O||N||A||L|| ||M||O||R||P||H||O||L||O||G||Y|
                                     2.|P||S||E||P||H||O||L||O||G||Y|
                                              3.|S||E||L||E||N||O||L||O||G||Y|
                                                    4.|E||L||E||C||T||R||O||S||T||A||T||I||C||S|
                                                    5.|C||L||I||M||A||T||O||L||O||G||Y|
                                                 6.|Q||U||A||N||T||U||M|| ||F||I||E||L||D|| ||T||H||E||O||R||Y|
                               7.|P||T||E||R||I||D||O||L||O||G||Y|
                                  8.|R||E||G||I||O||N||A||L|| ||A||N||A||T||O||M||Y|
                                              9.|V||E||R||T||E||B||R||A||T||E|| ||P||A||L||E||O||N||T||O||L||O||G||Y|
                      10.|S||T||R||U||C||T||U||R||A||L|| ||G||E||N||O||M||I||C||S|
                                    11.|M||A||C||R||O||B||

In [11]:
solved_crossword_list = []
graphical_crossword_list = []
print("   " * (left_size + 2) + ' 0.')
for index in range(0, len(words)):
    character = column_word[index]
    word = words[index].lower()
    meeting_center_index = word.find(character)
    if index < 10:
        row_representation = "   " * (left_size - meeting_center_index + 1) + ' ' + str(index + 1) + '.'
    else: 
        row_representation = "   " * (left_size - meeting_center_index + 1) + str(index + 1) + '.'
    for letter in word.upper():
        row_representation +='| |'
    print(row_representation)
    graphical_crossword_list.append(row_representation)
    
print('\n\n')
print('Clues:')

print('0. ' + column_clue)
for index in range (0, len(clues)):
    given_word = input('{0}. {1}'.format(str(index + 1), clues[index]))
    if given_word.lower() == words[index].lower():
        row_representation = graphical_crossword_list[index].split('.')[0]
        for letter in given_word.upper():
            row_representation += '|{0}|'.format(letter)
        solved_crossword_list.append(row_representation)


print('\n\n')
print("   " * (left_size + 2) + ' 0.')
for row in solved_crossword_list:
    print(row)

                                                       0.
             1.| || || || || || || || || || || || || || || || || || || || || || || |
                                     2.| || || || || || || || || || |
                                              3.| || || || || || || || || || |
                                                    4.| || || || || || || || || || || || || || |
                                                    5.| || || || || || || || || || || |
                                                 6.| || || || || || || || || || || || || || || || || || || || |
                               7.| || || || || || || || || || || |
                                  8.| || || || || || || || || || || || || || || || |
                                              9.| || || || || || || || || || || || || || || || || || || || || || || |
                      10.| || || || || || || || || || || || || || || || || || || |
                                    11.| || || || || || ||

1. The part of grammar that deals with the derivations of words. derivational morphology
2. The branch of sociology that studies election trends (as by opinion polls). psephology
3. The branch of astronomy that deals with the moon. selenology
4. The branch of physics that deals with static electricity. electrostatics
5. Meteorology of climates and their phenomena. climatology
6. The branch of quantum physics that is concerned with the theory of fields; it was motivated by the question of how an atom radiates light as its electrons jump from excited states. quantum field theory
7. The branch of botany that studies ferns. pteridology
8. Find a synonym for topology. regional anatomy
9. The paleontology of vertebrates. vertebrate paleontology
10. The branch of genomics that determines the three-dimensional structures of proteins. structural genomics
11. The theory of promoting health and longevity by means of diet (especially whole beans and grains). macrobiotics
12. Branch of optics deali




                                                       0.
             1|D||E||R||I||V||A||T||I||O||N||A||L|| ||M||O||R||P||H||O||L||O||G||Y|
                                     2|P||S||E||P||H||O||L||O||G||Y|
                                              3|S||E||L||E||N||O||L||O||G||Y|
                                                    4|E||L||E||C||T||R||O||S||T||A||T||I||C||S|
                                                    5|C||L||I||M||A||T||O||L||O||G||Y|
                                                 6|Q||U||A||N||T||U||M|| ||F||I||E||L||D|| ||T||H||E||O||R||Y|
                               7|P||T||E||R||I||D||O||L||O||G||Y|
                                  8|R||E||G||I||O||N||A||L|| ||A||N||A||T||O||M||Y|
                                              9|V||E||R||T||E||B||R||A||T||E|| ||P||A||L||E||O||N||T||O||L||O||G||Y|
                      10|S||T||R||U||C||T||U||R||A||L|| ||G||E||N||O||M||I||C||S|
                                    11|M||A||C||R||O||B||I||O||T|